In [1]:
import sys
import os
sys.path.append("..")
from tqdm import tqdm
import numpy as np
from utils import create_metadata_df
from dataset_xy import CARLADatasetXY
from data_preprocessing import transform_lidar_bev

from PIL import Image
import cv2


/Users/julianvonklitzing/miniforge3/envs/carla/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
path_data = os.path.join("..", "..","data", "data")
path_data = os.path.join("..", "..","data", "Noise-Dataset")

config_xy = {"used_inputs": ["lidar", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "y": ["brake", "steer", "throttle"],
        "seq_len": 1
        }

# Create df_meta 
df_meta_data = create_metadata_df(path_data, config_xy["used_inputs"])
df_meta_data = df_meta_data.sort_values(by=["dir", "measurements"]).reset_index(drop=True)

# Create Dataset & DataLoader
dataset = CARLADatasetXY(root_dir=path_data, df_meta_data=df_meta_data, config=config_xy)

In [6]:
df_meta_data

,dir,lidar,measurements
0,../../data/Noise-Dataset/Noise-Dataset1/Town01...,0010.npy,0010.json
1,../../data/Noise-Dataset/Noise-Dataset1/Town01...,0011.npy,0011.json
2,../../data/Noise-Dataset/Noise-Dataset1/Town01...,0012.npy,0012.json
3,../../data/Noise-Dataset/Noise-Dataset1/Town01...,0013.npy,0013.json
4,../../data/Noise-Dataset/Noise-Dataset1/Town01...,0014.npy,0014.json
...,...,...,...
11835,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0035.npy,0035.json
11836,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0036.npy,0036.json
11837,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0037.npy,0037.json
11838,../../data/Noise-Dataset/Noise-Dataset2/Town07...,0038.npy,0038.json


In [7]:
df_meta = dataset.df_meta_data
for idx in tqdm(range(len(df_meta))):
    dir_new = os.path.join(dataset.df_meta_data["dir"][idx], "lidar_bev")
    if not os.path.exists(dir_new):
        os.makedirs(dir_new)
    path_load = os.path.join(df_meta.iloc[idx][0], "lidar", df_meta.iloc[idx]["lidar"])
    lidar_np = dataset.load_data_from_path(path_load)
    lidar_transformed_np = transform_lidar_bev(lidar_np)
    lidar_img = Image.fromarray(lidar_transformed_np)
    path_save = os.path.join(dir_new, f"{df_meta.iloc[idx]['lidar'].split('.')[0]}{'.png'}")
    cv2.imwrite(path_save, lidar_transformed_np)

100%|██████████| 11840/11840 [00:58<00:00, 201.02it/s]
